<h1>Predicting game prices</h1>
<h5 style="margin-left: 2rem">By: Elad Ben-Haim, Shalev Hadar</h5>
<br/>
<br/>

<h4>נושא המחקר</h4>
<table dir="rtl">
    <tr>
        <th>
            נושא המחקר
        </th>
        <th>
            פירוט הנושא
        </th>
    </tr>
    <tr>
        <td>
        האם ניתן לחזות מחיר של משחק בעוד x זמן?
        </td>
        <td>
        מתי הכי ישתלם לקנות את המשחק בעתיד
        </td>
    </tr>
    <tr>
        <td>
        האם ניתן לחזור מתי למוכר הכי משתלם לעשות מבצע על המשחק?
        </td>
        <td>
        מתי הכי כדאי למוכר לעשות מבצע כדי להביא עוד שחקנים ולהמשיך למכור עם הרווח הגדול ביותר
        </td>
    </tr>
</table>
<br/>

<br/>

<h4>נתוני המחקר ודרכי ניתוח</h4>
<table dir="rtl">
    <tr>
        <th>
            נתוני המחקר & דרכי ניתוח
        </th>
        <th>
             ואיך ננתח אותם
        </th>
    </tr>
    <tr>
        <td>
		פירוט הנתונים: פרטים פיננסיים הוא נמכר בזמן X ואז לחזות בכמה הוא ימכר בזמן Y ובנוסף פרטים על המשחק - כמו שם, ז'אנר פופולריות וכו'.
        </td>
        <td>
            <p style="font-size: 1.1rem">
                נשתמש ב-Crawling על אתר isThereAnyDeal(Fig.3)<br/>
                כדי לא לקבל הודעת שגיאה על שימוש יתר, נשתמש ב-PROXY כדי לא להחסם ע"י isThereAnyDeal<br/>
                ונייבא משם את כל המידע הדרוש כדי לחזות את המחיר של משחק בעוד X זמן מסוים.<br/>
                ראינו שלכל משחק קיים באיזור ה1000+ (Fig.2) רשומות של log (Fig.1) של המחיר שלו ביחס לזמן, ולחנות בה הוא נמכר ואת העלייה\ירידה במחיר של המשחק ביחד ללוג הקודם.<br/>
                בעזרת STEAM API נוציא את ז'אנר המשחק, שנת הייצור ועוד פרטים מורכבים יותר על המשחק עצמו<br/>
                ולבסוף נצרף לכל לוג את פרטי המשחק ונקבל Dataset בגודל n = כמות המשחקים, x = כמות הלוגים, כלומר n*x<br/>
                אנחנו מעוניינים כרגע לקחת את 100 המשחקים הראשונים ולפי מה שראינו כמות הלוגים בדרך כלל היא לפחות 1000 אז נקבל בסביבות ה100,000+ רשומות.<br/>
            </p>
        </td>
    </tr>
    <tr>
        <td>
		דרכי ניתוח: נשתמש בכלים שלמדנו במהלך הקורס לעבד\ללמוד מהנתונים כמה מידע שאנו צריכים למטרה זו
        </td>
        <td>
            <p style="font-size: 1.1rem">
                ננתח את הDataFrame, בעזרת טבלאות יחסי משתנים, סטטיסטיקות, ולבסוף ננסה ללמד מכונה שתחזה את התאריך של המחיר הזול ביותר בשנה מסוימת, ואת המחיר אשר יביא את כמות המכירות הגדול ביותר
            </p>
        </td>
    </tr>
</table>
<br/>
<table style="width:100%;grid-template-rows: 1fr 1fr 1fr;">
<tr>
    <td>
        <figure>
            <img src="images\log_table_for_ds3.png" alt="Is there any deal log table">
            <figcaption>Fig.1 - The main crawled data source</figcaption>
        </figure>
    </td>
    <td>
        <figure>
            <img src="images\number_of_logs_for_ds3.png" alt="Example for number of rows in a typical game">
            <figcaption>Fig.2 - Example for number of rows in a typical game (Dark souls 3) </figcaption>
        </figure>
    </td>
    <td>
        <figure>
            <img src="images\is_there_any_deal_site_example_ds3.png" alt="Is there any deal game page">
            <figcaption>Fig.3 - IsThereAnyDeal game page</figcaption>
        </figure>
    </td>
</tr>

<h4>Importing</h6>

In [52]:
import requests
import bs4
from bs4 import BeautifulSoup
import random
import itertools
import re
import pandas as pd
from pandas import DataFrame
import time
import datetime
import os.path



<h4>Global functions</h4>

In [7]:
def get_html_response(url: str, proxy: str = None, params: list = None) -> requests.Response:
    time.sleep(1)
    if (proxy is not None):
        return requests.get(url, proxies={"http": proxy, "https":proxy}, params=params)
    else:
        return requests.get(url, params=params)

def get_response_as_beautiful_soup(req: requests.Response) -> BeautifulSoup:
    return BeautifulSoup(req.text, 'html.parser')

<br/>

<h4>Defining proxies for scraping</h4>

<h5>Get proxy list response html website</h5>
<h6>get the html as response object instead of getting the html again and again</h6>

In [8]:
# Get the html of the proxy list website
def get_proxy_list_html() -> requests.Response:
    # Website to get free proxies
    return get_html_response('https://free-proxy-list.net/')

In [9]:
proxies_response = get_proxy_list_html()

<h5>Scrape proxy ip addresses</h5>
<h6>gets the ip addresses as a list, shuffles them and returns an iterator to cycle through when making scrape requests</h6>

In [10]:
def get_proxy_list() -> itertools.cycle:
    soup = get_response_as_beautiful_soup(proxies_response)
    proxy_soup_list = soup.select('#list > div > div.table-responsive > div > table > tbody > tr')
    proxy_list = list(map(lambda i: i.select('td:nth-child(1)')[0].text + ':' + i.select('td:nth-child(2)')[0].text, proxy_soup_list))
    length = len(proxy_list)
    random.shuffle(proxy_list)
    return itertools.cycle(proxy_list), length

In [11]:
proxy_list, proxy_list_length = get_proxy_list()
current_proxy = next(proxy_list)

In [12]:
def get_proxied(url: str, params: list = None) -> requests.Response:
    i = 0
    while(i < proxy_list_length / 2):
        try:
            response = get_html_response(url, current_proxy, params=params)
            time.sleep(2)
            return response
        except:
            i += 1
        finally:
            current_proxy = next(proxy_list)

    raise RuntimeError('Half of the proxies provided don`t work.')

<br/>

<h4>Scrape isThereAnyDeal website</h4>
<h6>Steps:</h6>
<ol>
<li>Crawl list of top 100 trending games</li>
<li>For each game:</li>
<ul>
    <li>get the game details from steam API using "appId" scraped either from PC Gaming Wiki or Steam Ladder links</li>
    <li>mine price Log history on isThereAnyDeal</li>
    <li>mine Number of sales of the game</li>
<ul>
</ol>

In [13]:
is_there_any_deal_url = 'https://isthereanydeal.com'
steam_api_url = 'https://store.steampowered.com/api/appdetails'

<h5>Get list of 100 top trending games</h5>

In [14]:
def get_is_there_any_deal_games_response() -> requests.Response:
    #filteredUrl = 'https://isthereanydeal.com/?by=trending:desc#/filter:&pl/windows,&drm/steam,steam,-dlc,-type/6,-type/8,-type/7,&releaseyear/2015/2020;/options:all'
    filteredUrl = 'https://isthereanydeal.com/'
    return get_html_response(filteredUrl)

In [15]:
is_there_any_deal_games_response = get_is_there_any_deal_games_response()
print(is_there_any_deal_games_response)

<Response [200]>


In [16]:
def add_game_to_dataframe(df: DataFrame, game: dict) -> DataFrame:
    return df.append(DataFrame([game]), ignore_index=True)

In [17]:
def get_steam_api_game_response(steamId: str) -> requests.Response:
    return get_html_response(steam_api_url, params={'appids': steamId})

In [18]:
def delete_if_exists(d: dict, *keys: list) -> dict:
    for key in keys:
        if d.get(key) is not None:
            del d[key]
    return d

In [19]:
def get_steam_api_game_details(steamId: str) -> dict:
    data = get_steam_api_game_response(steamId).json()
    if data.get(steamId) is None or data.get(steamId).get('data') is None:
        return {}
    data = data[steamId]['data']

    if data.get('categories') is not None:
        for category in data['categories']:
            data['category.'+str(category['id'])]=True
        del data['categories']
    
    if data.get('genres') is not None:
        for genre in data['genres']:
            data['genre.'+str(genre['id'])]=True
        del data['genres']
    
    if data.get('developers') is not None:
        for developer in data['developers']:
            data[f'developer.' + developer.strip().replace(' ', '_')]=True
        del data['developers']
    
    if data.get('publishers') is not None:
        for publisher in data['publishers']:
            data[f'publisher.' + publisher.strip().replace(' ', '_')]=True
        del data['publishers']
        
    if data.get('supported_languages') is not None:
        for language in data['supported_languages']\
                            .replace('<strong>*</strong>', '')\
                            .replace('<br/>', '')\
                            .replace('<br>', '')\
                            .replace('languages with full audio support', '')\
                            .split(','):
            data[f'supported_language.' + language.strip().replace(' ', '_')]=True
        del data['supported_languages']

    return data

<img style="width: 50%" src="images/steam_api_response.png"/>

In [20]:
def get_game_dataframe():
    soup = get_response_as_beautiful_soup(is_there_any_deal_games_response)
    df = DataFrame()
    games = soup.select("#games > div.game")
    game_dict = dict()
    for game in games:
        steamId = game.attrs.get('data-steamid')
        if steamId is not None:
            steamId: str
            if 'app' in steamId:
                steamId = steamId.split('/')[1]
                title = game.select("div.title > a")[0].text
                history = game.select("div.overview.exp.tgl-hide > a:nth-child(5)")[0].attrs.get('href')
                game_details = get_steam_api_game_details(steamId)
                if game_details == {}: continue
                df = add_game_to_dataframe(df, {'steamId': steamId, 'title': title, 'history_link': is_there_any_deal_url + history, **game_details})
    return df

In [21]:
games_dataframe = get_game_dataframe()
games_dataframe.to_csv('games_dataframe.csv')
games_dataframe

KeyboardInterrupt: 

In [22]:
games_dataframe = pd.read_csv('./games_dataframe.csv')

In [23]:
def drop_columns(df: DataFrame, columns) -> DataFrame:
    return df.drop(columns, axis=1)

In [24]:

def map_steam_api_game_data(ndf: DataFrame) -> DataFrame:
    df = ndf.copy()
    ignored_columns = [
        'type',
        'name',
        'steam_appid',
        'detailed_description',
        'about_the_game',
        'short_description',
        'fullgame',
        'header_image',
        'website',
        'pc_requirements',
        'mac_requirements',
        'linux_requirements',
        'legal_notice',
        'price_overview',
        'package_groups',
        'screenshots',
        'achievements',
        'background',
        'content_descriptors',
        'support_info',
        'ext_user_account_notice',
        'reviews',
        'drm_notice'
    ]
    df = drop_columns(df, ignored_columns)
    df['number_of_demos'] = df['demos'].apply(lambda demo: len(demo) if type(demo) is list else None)
    df = drop_columns(df, 'demos')

    df['num_of_game_videos'] = df['movies'].apply(lambda movies: len(movies) if type(movies) is list else None)
    df = drop_columns(df, 'movies')

    df['num_of_packages_game_is_in'] = df[df['packages'].notnull()].apply(lambda packages: len(packages) if type(packages) is list else None)
    df = drop_columns(df, 'packages')

    df['metacritic_score'] = df['metacritic'].apply(lambda metacritic : metacritic['score'] / 100 if type(metacritic) is dict else None)
    df = drop_columns(df, 'metacritic')

    df['windows_supported'] = df['platforms'].apply(lambda platforms: platforms['windows'] == True if type(platforms) is dict else False)
    df['mac_supported'] = df['platforms'].apply(lambda platforms: platforms['mac'] == True if type(platforms) is dict else False)
    df['linux_supported'] = df['platforms'].apply(lambda platforms: platforms['linux'] == True if type(platforms) is dict else False)
    df = drop_columns(df, 'platforms')
    
    df['total_steam_recommendations'] = df['recommendations'].apply(lambda recommendations : recommendations['total'] if type(recommendations) is dict else None)
    df = drop_columns(df, 'recommendations')

    df['release_date'] = df['release_date'].apply(lambda release_date : release_date['date'] if type(release_date) is dict else None)
    df = drop_columns(df, 'release_date')

    df['num_of_dlc'] = df['dlc'].apply(lambda dlc: len(dlc) if type(dlc) is list else 0)
    df = drop_columns(df, 'dlc')
    return df

In [25]:
games_dataframe2 = map_steam_api_game_data(games_dataframe)
games_dataframe2.to_csv('games_details_data.csv')

<h5>Get price history for each game</h5>

In [59]:
def get_game_history_details(steamId: str, history_link: str) -> DataFrame:
    if os.path.isfile(f'games/{steamId}.html'):
        f = open(f'games/{steamId}.html', 'r')
        soup = BeautifulSoup(f.read(), 'html.parser')
        f.close()
    else:
        res = get_html_response(history_link)
        f = open(f'games/{steamId}.html', 'a')
        f.write(res.text)
        f.close()
        soup = get_response_as_beautiful_soup(res)
    
    df = DataFrame()
    vals = soup.select("div.lg2.game")
    for val in vals:
        dateText = val.select_one("span.lg2__time-rel").attrs['title']
        date = datetime.datetime.strptime(dateText, '%a, %d %b %Y %H:%M:%S +0000')
        shop_title = val.select_one('.shopTitle').text.strip()
        regular_price = val.select_one('div:nth-child(2) > span.lg2__price').text.replace('$', '').strip()
        price_now = val.select_one('div:nth-child(3) > span.lg2__price').text.replace('$', '').strip()
        price_change_percentage = val.select_one('div:nth-child(3) > span.lg2__change').text.replace('$','').replace('%', '')
        if price_change_percentage.startswith('-') and price_change_percentage.replace('-', '') != '':
            price_change_percentage = float(price_change_percentage.replace('-', '')) / 100
        elif price_change_percentage.startswith('+') and price_change_percentage.replace('+', '') != '':
            price_change_percentage = (float(price_change_percentage.replace('+', '')) / 100) + 1
        df = df.append(DataFrame([{'record_date': date, 'shop': shop_title, 'regular_price': regular_price, 'price_now': price_now, 'price_change': price_change_percentage}]))
    return df


In [64]:
print(games_dataframe2)
games_with_history = DataFrame()
for index, row in games_dataframe2.iterrows():
    row_df = DataFrame([row])
    game_history_details = get_game_history_details(row['steamId'], row['history_link'])

    game_history_details['key'] = 1
    row_df['key'] = 1

    cross = row_df.merge(game_history_details, how = 'outer')
    games_with_history = games_with_history.append(cross)

games_with_history

    Unnamed: 0  steamId  ... total_steam_recommendations num_of_dlc
0            0   812140  ...                        None          0
1            1  1174180  ...                        None          0
2            2  1448030  ...                        None          0
3            3   788100  ...                        None          0
4            4  1091500  ...                        None          0
..         ...      ...  ...                         ...        ...
73          73   686810  ...                        None          0
74          74  1426300  ...                        None          0
75          75  1222670  ...                        None          0
76          76    34900  ...                        None          0
77          77  1184370  ...                        None          0

[78 rows x 278 columns]


KeyError: "['key'] not found in axis"

In [1]:
def save_game_as_csv(steamId: str, df: DataFrame):
    df.to_csv(f"games/steam_id_{steamId}")

NameError: name 'pd' is not defined

In [ ]:
def get_game_details():
    soup = get_response_as_beautiful_soup(game_details_response)
    return soup.select("#pageContainer > script:nth-child(9)")

gamedetails = str(get_game_details()[0])
m = re.findall('JSON\.stringify\((.*?)\)', gamedetails)
# print(m)

<h3>Used Resources</h3>
<dl>
    <dt>Scraping</dt>
    <dd>
        <a href="https://isthereanydeal.com/game/reddeadredemptionii/info/">
            <b>Is-There-Any-Deal website</b> For scraping cost history and more financial details
        </a>
    </dd>
    <dd>
        <a href="https://www.geeksforgeeks.org/web-scraping-without-getting-blocked/">
            <b>Using Proxies to avoiding getting blocked</b>
        </a>
    </dd>
    <dd>
        <a href="https://wiki.teamfortress.com/wiki/User:RJackson/StorefrontAPI#App_info">
            <b>Steam StoreFront API</b> Limited to 100,000 requests per day, and no more than 10 per second
        </a>
    </dd>
    <dd>
        <a href="https://store.steampowered.com/api/appdetails">Steam API for Game Metadata - https://store.steampowered.com/api/appdetails?appids=1091500</a>
    </dd>
</dl>